In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers

def prepare_data(data_file_name):
    """
    Responsible for cleaning the data file provided from the UCI machine
    learning repository here: http://archive.ics.uci.edu/ml/datasets/Mushroom.
    The function then produces two CSV files appropriately formatted to be
    used in TensorFlow where the CSV files split with respect to
    training and testing data.
    """

    # The header is formed from the 'agaricus-lepiota.name' file found on
    # http://archive.ics.uci.edu/ml/datasets/Mushroom
    header = ['class', 'cap_shape', 'cap_surface',
              'cap_color', 'bruises', 'odor', 'gill_attachment',
              'gill_spacing', 'gill_size', 'gill_color', 'stalk_shape',
              'stalk_root', 'stalk_surface_above_ring',
              'stalk_surface_below_ring', 'stalk_color_above_ring',
              'stalk_color_below_ring', 'veil_type', 'veil_color',
              'ring_number', 'ring_type', 'spore_print_color',
              'population', 'habitat']
    df = pd.read_csv(data_file_name, sep=',', names=header)

    # Entries with a '?' indicate a missing piece of data, and
    # these entries are dropped from our dataset.
    df.replace('?', np.nan, inplace=True)
    df.dropna(inplace=True)

    # The class of poisonous or edible is indicated in the data as
    # either 'p' or 'e' respectively. We require that this is numeric,
    # and therefore use '0' to indicate poisonous (or not edible) and
    # '1' to indicate edible.
    df['class'].replace('p', 0, inplace=True)
    df['class'].replace('e', 1, inplace=True)

    # Since we are dealing with non-numeric feature data, or in other
    # words, categorical data, we need to replace these with numerical
    # equivalents. Pandas has a nice function called "get_dummies" that
    # performs this task.
    cols_to_transform = header[1:]
    df = pd.get_dummies(df, columns=cols_to_transform)

    # We can now split the data into two separate data frames,
    # one for training, which will constitute the bulk of the
    # data, and one for testing.
    df_train, df_test = train_test_split(df, test_size=0.1)

    # Determine the number of rows and columns in each of the
    # data frames.
    num_train_entries = df_train.shape[0]
    num_train_features = df_train.shape[1] - 1

    num_test_entries = df_test.shape[0]
    num_test_features = df_test.shape[1] - 1

    # The data frames are written as a temporary CSV file, as we still
    # need to modify the header row to include the number of rows and
    # columns present in the training and testing CSV files.
    df_train.to_csv('mushroom_train.csv', index=False)
    df_test.to_csv('mushroom_test.csv', index=False)

    
MUSHROOM_DATA_FILE = "agaricus-lepiota.data"

# Prepare the mushroom data for TensorFlow by
# creating two train / test CSV files.
prepare_data(MUSHROOM_DATA_FILE)

In [2]:
df_train = pd.read_csv('mushroom_train.csv')
print(df_train.head())

train_label = np.array(df_train['class'])
train_data =  np.array(df_train.drop('class', axis=1))

   class  cap_shape_b  cap_shape_c  cap_shape_f  cap_shape_k  cap_shape_s  \
0      0            0            0            0            0            0   
1      0            0            0            0            0            0   
2      0            0            0            1            0            0   
3      1            0            0            0            0            0   
4      1            0            0            1            0            0   

   cap_shape_x  cap_surface_f  cap_surface_g  cap_surface_s  ...  \
0            1              0              0              0  ...   
1            1              1              0              0  ...   
2            0              1              0              0  ...   
3            1              1              0              0  ...   
4            0              0              0              0  ...   

   population_n  population_s  population_v  population_y  habitat_d  \
0             0             0             0             

In [3]:
df_test = pd.read_csv('mushroom_test.csv')
print(df_test.head())
test_label = np.array(df_test['class'])
test_data =  np.array(df_test.drop('class', axis=1))

   class  cap_shape_b  cap_shape_c  cap_shape_f  cap_shape_k  cap_shape_s  \
0      0            0            0            0            0            0   
1      1            0            0            1            0            0   
2      1            0            0            1            0            0   
3      0            0            0            1            0            0   
4      1            0            0            1            0            0   

   cap_shape_x  cap_surface_f  cap_surface_g  cap_surface_s  ...  \
0            1              0              0              0  ...   
1            0              1              0              0  ...   
2            0              1              0              0  ...   
3            0              0              0              0  ...   
4            0              1              0              0  ...   

   population_n  population_s  population_v  population_y  habitat_d  \
0             0             0             0             

In [4]:
print(train_data.shape, ' ', train_label.shape)
print(test_data.shape, ' ', test_label.shape)

(5079, 98)   (5079,)
(565, 98)   (565,)


In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
tf.compat.v1.reset_default_graph()

(x_train, y_train), (x_test, y_test) = (train_data , train_label) , (test_data , test_label)
x_train = x_train.reshape([x_train.shape[0], -1])
x_test = x_test.reshape([x_test.shape[0], -1])
print(x_train.shape, ' ', y_train.shape)
print(x_test.shape, ' ', y_test.shape)

(5079, 98)   (5079,)
(565, 98)   (565,)


In [12]:
## build model
## write your code here
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
tf.compat.v1.reset_default_graph()

(x_train, y_train), (x_test, y_test) = (train_data , train_label) , (test_data , test_label)
x_train = x_train.reshape([x_train.shape[0], -1])
x_test = x_test.reshape([x_test.shape[0], -1])
print(x_train.shape, ' ', y_train.shape)
print(x_test.shape, ' ', y_test.shape)

model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(98,)),
    layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.L1(0.01)),
    layers.Dropout(0.2),
    layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.L1(0.01)),
    layers.Dropout(0.2),
    layers.Dense(2, activation='softmax', kernel_regularizer=tf.keras.regularizers.L1(0.01))
])


#keras.optimizers.Adagrad(learning_rate=0.01)
#keras.optimizers.Adam(learning_rate=0.01)
#keras.optimizers.RMSprop(learning_rate=0.01)

# provide labels as one_hot representation => tf.keras.losses.CategoricalCrossentropy
# provide labels as integers => tf.keras.losses.SparseCategoricalCrossentropy 
model.compile(optimizer=keras.optimizers.Adam(),
             loss=keras.losses.SparseCategoricalCrossentropy(),
             metrics=['accuracy'])
model.summary()

(5079, 98)   (5079,)
(565, 98)   (565,)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                6336      
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 130       
Total params: 14,786
Trainable params: 14,786
Non-trainable params: 0
____________

In [13]:
## training 
## write your code here
history = model.fit(x_train, y_train, batch_size=64, epochs=10, validation_split=0.3, verbose=2)

Epoch 1/10
56/56 - 0s - loss: 7.5368 - accuracy: 0.8759 - val_loss: 5.4577 - val_accuracy: 0.9895
Epoch 2/10
56/56 - 0s - loss: 4.0241 - accuracy: 0.9826 - val_loss: 2.6606 - val_accuracy: 0.9961
Epoch 3/10
56/56 - 0s - loss: 1.7800 - accuracy: 0.9854 - val_loss: 0.9794 - val_accuracy: 0.9974
Epoch 4/10
56/56 - 0s - loss: 0.6412 - accuracy: 0.9899 - val_loss: 0.4005 - val_accuracy: 0.9974
Epoch 5/10
56/56 - 0s - loss: 0.3922 - accuracy: 0.9851 - val_loss: 0.3230 - val_accuracy: 0.9993
Epoch 6/10
56/56 - 0s - loss: 0.3398 - accuracy: 0.9904 - val_loss: 0.2892 - val_accuracy: 1.0000
Epoch 7/10
56/56 - 0s - loss: 0.3144 - accuracy: 0.9882 - val_loss: 0.2663 - val_accuracy: 1.0000
Epoch 8/10
56/56 - 0s - loss: 0.2941 - accuracy: 0.9885 - val_loss: 0.2481 - val_accuracy: 1.0000
Epoch 9/10
56/56 - 0s - loss: 0.2781 - accuracy: 0.9924 - val_loss: 0.2348 - val_accuracy: 1.0000
Epoch 10/10
56/56 - 0s - loss: 0.2605 - accuracy: 0.9916 - val_loss: 0.2230 - val_accuracy: 1.0000


In [16]:
## save model
## write your code here
model.save('DNN_mushroom.h5')

In [17]:
## load model and use test data to evaluate result
result = model.evaluate(x_test, y_test)
print(result)

18/18 [==============================] - 0s 388us/step - loss: 0.2220 - accuracy: 1.0000
[0.22203372418880463, 1.0]
